<a href="https://colab.research.google.com/github/pilipandr770/Coments_model/blob/main/Coment_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install pandas
!pip install scikit-learn


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import json

# Створення директорії, якщо вона не існує
os.makedirs('/root/.kaggle/', exist_ok=True)

# Створення та збереження kaggle.json
kaggle_data = {"username":"andriipylypchuk","key":"cc0b80cb1380b338605d4818e95ccbec"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(kaggle_data, file)

# Налаштування дозволів на файл
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification


100% 1.08G/1.08G [00:59<00:00, 19.8MB/s]
100% 1.08G/1.08G [00:59<00:00, 19.5MB/s]


In [ ]:
!unzip /content/jigsaw-multilingual-toxic-comment-classification.zip -d /content/


Archive:  /content/jigsaw-multilingual-toxic-comment-classification.zip
  inflating: /content/jigsaw-toxic-comment-train-processed-seqlen128.csv  
  inflating: /content/jigsaw-toxic-comment-train.csv  
  inflating: /content/jigsaw-unintended-bias-train-processed-seqlen128.csv  
  inflating: /content/jigsaw-unintended-bias-train.csv  
  inflating: /content/sample_submission.csv  
  inflating: /content/test-processed-seqlen128.csv  
  inflating: /content/test.csv       
  inflating: /content/test_labels.csv  
  inflating: /content/validation-processed-seqlen128.csv  
  inflating: /content/validation.csv  


In [ ]:
import pandas as pd

# Завантажимо тренувальний файл
train_data = pd.read_csv('/content/jigsaw-toxic-comment-train.csv')
print(train_data.head())


                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [ ]:
print(train_data.isnull().sum())


id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [ ]:
comments = train_data['comment_text']
labels = train_data['toxic']


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Токенізація текстів
inputs = tokenizer(comments.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=128)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
from transformers import BertTokenizer

# Ініціалізація токенізатора BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Токенізація тексту коментарів (перевірте, що 'comments' містить текст)
inputs = tokenizer(comments.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=128)

# Отримуємо токенізовані входи
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']


In [13]:
import torch
from torch.utils.data import TensorDataset

# Припустимо, що train_data вже завантажено і мітки в стовпці 'toxic'
labels = train_data['toxic']

# Перетворюємо labels на тензор
labels = torch.tensor(labels.values)

# Створюємо TensorDataset
dataset = TensorDataset(input_ids, attention_mask, labels)


In [14]:
from transformers import BertForSequenceClassification, AdamW

# Завантажте модель BERT для класифікації (2 вихідні класи)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Оптимізатор (AdamW є рекомендованим для BERT)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
from transformers import BertTokenizer

# Токенізуємо текст
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
inputs = tokenizer(comments.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=128)

# Отримуємо входи для моделі
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split

# Розподіл даних на тренувальні та валідаційні набори
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1)
train_masks, val_masks = train_test_split(attention_mask, test_size=0.1)


In [ ]:
from torch.utils.data import TensorDataset

# Створюємо TensorDataset для тренування
train_data = TensorDataset(train_inputs, train_masks, train_labels)
val_data = TensorDataset(val_inputs, val_masks, val_labels)


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

# Створюємо DataLoader для тренування
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Створюємо DataLoader для валідації
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


In [19]:
from transformers import BertForSequenceClassification, AdamW

# Завантажуємо модель BERT для класифікації (2 вихідні класи)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Оптимізатор
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/saved_tokenizer')

In [ ]:
# Збереження стану оптимізатора
torch.save(optimizer.state_dict(), '/content/drive/MyDrive/saved_optimizer.pth')


In [ ]:
import torch

# Збереження стану оптимізатора
torch.save(optimizer.state_dict(), '/content/drive/MyDrive/saved_optimizer.pth')


In [24]:
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/saved_optimizer.pth'))


<ipython-input-24-f69519e408b5>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  optimizer.load_state_dict(torch.load('/content/drive/MyDrive/saved_optimizer.pth'))
<ipython

In [25]:
from google.colab import drive
drive.mount('/content/drive')

# Після цього переконайтесь, що файл збережений у відповідній директорії на Google Drive.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
